In [32]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import string
import pymongo

In [15]:
base_url = 'https://www.rottentomatoes.com/m/'
# add the correct User-Agent
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}

In [19]:
# format url for movie with normal names
def get_url(base_url, movie_name):
    title = ''.join(item for item in movie_name if item not in string.punctuation)
    tag_list = title.lower().split()
    
    tag = ''
    for item in tag_list:
        if(item != tag_list[-1]):
            tag = tag + str(item) + '_'
        else:
            tag = tag + str(item)
    
    url = base_url + tag
    
    return url

In [17]:
movie_data = pd.read_csv('../movie_base.csv')
movie_data.head()

,Unnamed: 0,Rank,Released,Movie,DomesticBox Office,InternationalBox Office,WorldwideBox Office
0,0,1,2015,Star Wars Ep. VII: The Force Awakens,936662225,1116648,2053311
1,1,2,2009,Avatar,760507625,2015837,2776345
2,2,3,2018,Black Panther,700059566,648300000,1348359
3,3,4,2018,Avengers: Infinity War,678815482,1369988,2048803
4,4,5,1997,Titanic,659363944,1548844,2208208


In [21]:
# get url list waiting to crawl
tomato_url_list = []

for movie in movie_data.Movie:
    tomato_url_list.append(get_url(base_url, movie))

In [22]:
# request url
def get_html(url):
    try:
        r = requests.get(url, timeout=30)
        r.raise_for_status
        #site encoding
        r.encoding = 'utf-8'
        return r.text
    except:
        print('ERROR')
        pass

In [23]:
def get_content(url):
    html = get_html(url)
    soup = bs(html, 'lxml')
    data = {}
    try:
        try:
            data['cretic(%)'] = soup.find('span', class_='superPageFontColor').span.text
        except:
            data['cretic(%)'] = 'na'
            
        try:
            data['cretic_rate'] = soup.find('div', id='scoreStats').findChildren()[0].text[25:].strip()
        except:
            data['cretic_rate'] = 'na'
            
        try:
            data['cretic_count'] = soup.find('div', id='scoreStats').findChildren()[2].text[18:].strip()
        except:
            data['cretic_count'] = 'na'
            
        try:
            data['cretic_fresh'] = soup.find('div', id='scoreStats').findChildren()[7].text
        except:
            data['cretic_fresh'] = 'na'
        
        try:
            data['cretic_rot'] = soup.find('div', id='scoreStats').findChildren()[10].text
        except:
            data['cretic_rot'] = 'na'
            
        try:
            data['user(%)'] = soup.find('div', class_='audience-score').find('span', class_='superPageFontColor').text[:2]
        except:
            data['user(%)'] = 'na'
            
        try:
            data['user_rate'] = soup.find('div', class_='audience-info').findChildren()[0].contents[2].strip()
        except:
            data['user_rate'] = 'na'
        
        try:
            data['user_number'] = soup.find('div', class_='audience-info').findChildren()[2].contents[2].strip()
        except:
            data['user_number'] = 'na'

    except:
        print('error')
    
    return data

In [28]:
# get movie list data
movie_crawl_list = []
for url in tomato_url_list:
    data = get_content(url)
    movie_crawl_list.append(data)

In [39]:
tomato_data = pd.DataFrame(data=movie_crawl_list,
                           columns=["cretic(%)", "cretic_count", 'cretic_fresh', 'cretic_rate', 'cretic_rot',
                                   'user(%)', 'user_number', 'user_rate'])
tomato_data['name'] = movie_data['Movie']
tomato_data['url'] = tomato_url_list
tomato_data['year'] = movie_data['Released']

In [40]:
# save raw data to my mongoDB
client = pymongo.MongoClient('mongodb://localhost:27017/')
# create database
db = client.movie_db
# create colletions
col = db['rotten_tomato']

In [41]:
col.insert_many(tomato_data.to_dict("records"))